<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Configuration" data-toc-modified-id="Configuration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configuration</a></span></li><li><span><a href="#Import-and-load" data-toc-modified-id="Import-and-load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import and load</a></span></li></ul></div>

## Configuration

In [1]:
DATA_FILE = '../data/2019–2020/postprocessed/df_Rcleaned_train.csv'

## Import and load

In [28]:
library(car)
library(tidyverse)
library(hexbin)
library(mice)
library(nlme)
library(lme4)
library(lmerTest)

# Display more data in the Jupyter notebook
options(repr.matrix.max.cols=500, repr.matrix.max.rows=50)

In [29]:
df = read.csv(DATA_FILE, na.strings=c("", " ", "NA"))
dim(df)
head(df)

[1] 560  61

,X,PID,gender,race,dorm,life_satisfaction_t1,empathy,loneliness_t1,stress_t1,BFI_E,BFI_A,BFI_C,BFI_N,BFI_O,intl_student,family_income,life_satisfaction_t2,loneliness_t2,stress_t2,parent_education_highest,wellbeing_composite_t1,wellbeing_composite_t2,degree_in_UNION,degree_out_UNION,empathy_UNION,degree_in_INTIMATE,degree_out_INTIMATE,empathy_INTIMATE,degree_in_ACQUAINTANCE,degree_out_ACQUAINTANCE,empathy_ACQUAINTANCE,degree_in_CloseFrds,degree_out_CloseFrds,empathy_CloseFrds,degree_in_NegEmoSupp,degree_out_NegEmoSupp,empathy_NegEmoSupp,degree_in_PosEmoSupp,degree_out_PosEmoSupp,empathy_PosEmoSupp,degree_in_Responsive,degree_out_Responsive,empathy_Responsive,degree_in_EmpSupp,degree_out_EmpSupp,empathy_EmpSupp,degree_in_PosAff,degree_out_PosAff,empathy_PosAff,degree_in_NegAff,degree_out_NegAff,empathy_NegAff,degree_in_Gossip,degree_out_Gossip,empathy_Gossip,degree_in_Liked,degree_out_Liked,empathy_Liked,degree_in_StudyWith,degree_out_StudyWith,empathy_StudyWith
,<int>,<int>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>
1,1,1001,male,south_asian,Rinconada,4.833333,6.375,1.333333,2.5,6.5,6.0,5.5,3.5,6.5,0,70000,6.166667,2.000000,1.5,5,-0.05862651,0.83993983,3,0,NA,1,0,NA,1,0,NA,1,0,NA,0,0,NA,2,0,NA,0,0,NA,0,0,NA,1,0,NA,0,0,NA,1,0,NA,1,0,NA,1,0,NA
2,2,1047,female,other_or_mixed,Twain,5.333333,6.500,2.000000,2.5,5.5,6.5,7.0,3.0,6.5,0,210000,5.166667,2.000000,3.0,5,-0.44690440,-0.66253028,6,4,6.437500,4,4,6.43750,0,1,6.125,4,4,6.43750,4,3,6.541667,3,4,6.4375,2,2,6.5625,1,2,6.5625,3,4,6.43750,1,1,6.5,4,3,6.541667,1,2,6.000,2,1,6.6250
3,3,1078,female,east_asian,Loro,5.500000,6.250,1.333333,2.0,4.5,6.5,6.5,1.5,6.5,0,130000,5.500000,2.333333,3.0,6,0.61446613,-0.79801599,1,0,NA,1,0,NA,1,0,NA,1,0,NA,0,0,NA,1,0,NA,1,0,NA,0,0,NA,0,0,NA,0,1,5.0,0,0,NA,0,0,NA,0,0,NA
4,4,1097,male,east_asian,Otero,6.000000,5.000,2.000000,2.0,3.0,4.0,5.5,1.5,4.5,0,90000,5.833333,2.000000,2.5,6,0.22618824,-0.04126834,6,4,5.812500,3,1,5.62500,2,0,NA,2,1,5.62500,1,1,5.625000,1,1,6.6250,0,1,5.6250,0,1,4.8750,1,1,6.12500,0,0,NA,0,1,5.625000,0,1,6.000,3,1,6.6250
5,5,1105,female,white,Larkin,6.666667,5.750,2.000000,2.0,4.5,5.0,5.0,2.0,4.5,0,170000,6.666667,2.000000,2.0,4,0.48041399,0.64021289,6,9,5.847222,4,5,6.02500,2,2,5.875,3,4,5.81250,2,3,6.125000,2,2,5.3750,2,2,6.3125,1,4,6.1875,2,1,6.87500,1,0,NA,2,3,5.750000,1,1,5.125,4,2,6.1875
6,6,1110,female,white,Larkin,5.166667,6.750,2.000000,2.0,6.5,5.0,6.0,2.0,4.5,0,110000,6.833333,1.666667,2.0,5,-0.09159395,0.95635646,4,5,5.550000,4,4,5.40625,1,0,NA,3,4,5.40625,3,4,5.406250,3,2,6.0625,2,2,5.7500,3,1,6.0000,2,4,5.78125,0,0,NA,3,4,5.406250,4,1,6.000,3,4,5.4375


In [30]:
model_C = lm(life_satisfaction_t2 ~ life_satisfaction_t1, df)
summary(model_C)$coefficients

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),1.0228226,0.15714766,6.508672,1.692618e-10
life_satisfaction_t1,0.7655304,0.03037431,25.203219,4.188638e-94


In [42]:
frame = NULL
for (col in names(df)) {
    if (startsWith(col, 'degree')) {
        model = lm(as.formula(paste('life_satisfaction_t2 ~', col)), df)
        b = summary(model)$coefficients[2, 1]
        p = summary(model)$coefficients[2, 4]
        frame = rbind(frame, c(col, b, p))
    }
}
frame = as.data.frame(frame)
names(frame) = c('column', 'beta', 'p')
frame$beta = as.numeric(as.character(frame$beta))
frame$p = as.numeric(as.character(frame$p))
frame$p_adjusted = p.adjust(frame$p, method="holm")
frame

column,beta,p,p_adjusted
<fct>,<dbl>,<dbl>,<dbl>
degree_in_UNION,0.03491602,0.049697962,0.8448654
degree_out_UNION,0.03826154,0.097791212,1.0000000
degree_in_INTIMATE,0.05057935,0.038389709,0.7677942
degree_out_INTIMATE,0.05591758,0.102163446,1.0000000
degree_in_ACQUAINTANCE,0.04012626,0.354450633,1.0000000
degree_out_ACQUAINTANCE,-0.04196406,0.429243249,1.0000000
degree_in_CloseFrds,0.05763047,0.031686587,0.6654183
degree_out_CloseFrds,0.03760797,0.311505237,1.0000000
degree_in_NegEmoSupp,0.06589729,0.041066762,0.7677942
